In [11]:
from openalea.plantgl.all import *
from openalea.mtg import *
from oawidgets.plantgl import PlantGL

# Draw a leaf
A leaf can be a complex shape or just a simple triangle

In [12]:
def bezier_leaf():
    l = []
    l.append([Vector4(-0.00170203, 0.00487903, -0.00155362, 1),
              Vector4(-0.00946124, 0.0267487, 0.00857975, 1),
              Vector4(-0.0145598, 0.0310762, 0.0565383, 1),
              Vector4(-0.00422035, 0.0237428, 0.101953, 1)])
    l.append([Vector4(3.9604e-05, 0.0019996, 5.60056e-06, 1),
              Vector4(-0.00466331, 0.00703859, 0.0339818, 1),
              Vector4(-0.00868596, 0.00523895, 0.076457, 1),
              Vector4(0.00379859, 0.00943444, 0.154352, 1)])
    l.append([Vector4(-3.9604e-05, -0.0019996, -5.60056e-06, 1),
              Vector4(-0.00493527, 0.00263947, 0.0352765, 1),
              Vector4(-0.00867663, 0.00259947, 0.0760139, 1),
              Vector4(0.00447972, 0.00923447, 0.156651, 1)])
    l.append([Vector4(-0.00218137, -0.00475904, 0.000459084, 1),
              Vector4(-0.0120507, -0.0206578, 0.0115464, 1),
              Vector4(-0.0150292, -0.0230562, 0.0604107, 1),
              Vector4(-0.00608397, -0.0102688, 0.102558, 1)])
    
    matrix = Point4Matrix(4, 4)
    for i, row in enumerate(l):
        for j, pt in enumerate(row):
            pt = pt * 9
            pt.w = 1
            matrix[i, j] = pt
    
    leaf = BezierPatch(matrix, 4, 4)

    leaf.name = 'leaf'
    return leaf


In [13]:
leaf=bezier_leaf()

In [14]:
PlantGL(leaf)

Plot(antialias=3, axes=['x', 'y', 'z'], axes_helper=1.0, axes_helper_colors=[16711680, 65280, 255], background…

In [15]:
# list of points
points =  [(-1,-1,0),
           (1,-1,0),
           (1,1,0),
           (-1,1,0)]
# list of indices
indices = [(0, 1, 2),(2,3,0)]
# creating the geometry
simple_leaf = TriangleSet(pointList=points, indexList=indices)
PlantGL(simple_leaf)

Plot(antialias=3, axes=['x', 'y', 'z'], axes_helper=1.0, axes_helper_colors=[16711680, 65280, 255], background…

# Generate a tree (Cylinder + list of leaves)

In [16]:
tree = []
height = 30
def trunk(height=height):
    return Cylinder(radius=1, height=height)

PlantGL(trunk())

Plot(antialias=3, axes=['x', 'y', 'z'], axes_helper=1.0, axes_helper_colors=[16711680, 65280, 255], background…

In [17]:
from random import uniform
from math import cos, sin, radians, pi

def spherical(radius):
    theta = uniform(-pi, pi)
    phi = uniform(-pi/2, pi/2)
    return [radius * cos(theta) * cos(phi),
            radius * sin(theta) * cos(phi),
            radius *sin(phi)]
            
def leaves(nb_leaves=1000, height=30, radius=20):
    l = []
    
    rads = [0, 5, 15, 20]
    p1, p2, p3 = 0.1, 0.6, 0.3
    
    for i in range(nb_leaves):
        p = uniform(0,1)
        if p <= p1:
            rad = uniform(rads[0], rads[1])
        elif p1< p <=p2:
            rad = uniform(rads[1], rads[2])
        else:
            rad = uniform(rads[2], rads[3])

        position = spherical(rad)
        position[2] +=(height+radius)

        az, el, roll = uniform(-pi, pi), uniform(0, pi), uniform(-pi, pi)
        _leaf = Translated(position, EulerRotated (az, el, roll, simple_leaf))

        l.append(_leaf)

    return l

scene = Scene(leaves())
scene+=trunk()
PlantGL(scene)
        
        

            

    
    

Plot(antialias=3, axes=['x', 'y', 'z'], axes_helper=1.0, axes_helper_colors=[16711680, 65280, 255], background…

In [18]:
from openalea.mtg import MTG

In [19]:

canopy = Group(leaves())
tree_trunk = trunk()

In [20]:
g_tree = MTG()
tid=g.add_component(g.root, label='Tree')
stem_id = g.add_component(tid, label='Stem')
leaf_id = g.add_child(stem_id, edge_type='+', label='Leaf')
stem = g.node(stem_id)
leaf = g.node(leaf_id)

stem.geometry=tree_trunk
stem.is_green=False
leaf.geometry = canopy
leaf.is_green = True

stem.anchor_point = Vector3(0,0,0)
stem.heading=Vector3(0,0,1)
stem.visible_length = stem.geometry.height
stem.stem_diameter = stem.geometry.radius*2

In [21]:
from openalea.archicrop import display

In [30]:
step=100
scene, labels = display.build_scene([g_tree], position=[(i*step,0,0) for i in range(10)])

In [31]:
PlantGL(scene)

Plot(antialias=3, axes=['x', 'y', 'z'], axes_helper=1.0, axes_helper_colors=[16711680, 65280, 255], background…

In [32]:
from openalea.astk.sky_irradiance import sky_irradiance
from openalea.astk.sky_sources import sky_sources, caribu_light_sources
from alinea.caribu.data_samples import data_path
# irr = sky_irradiance()
# sun, sky = sky_sources(sky_type='clear_sky', sky_irradiance=irr, scale='ppfd')
# lights = caribu_light_sources(sun, sky)
lights = str(data_path('Turtle46soc.light'))

In [33]:
cs, raw, agg = illuminate(scene, light=lights, labels=labels)

In [34]:
mean_leaf_irradiance(agg)

,Energy,area,Irradiance
plant,,,
0,0.118858,0.4,0.297145
1,0.118504,0.4,0.296260
2,0.118420,0.4,0.296050
3,0.118532,0.4,0.296330
4,0.118560,0.4,0.296400
5,0.118466,0.4,0.296165
6,0.118540,0.4,0.296350
7,0.118478,0.4,0.296195
8,0.118484,0.4,0.296210
